In [1]:
# importing libraries
import cv2
import numpy as np
import time

# capturing or reading video
# cap = cv2.VideoCapture('./Car_data/video.avi')
cap = cv2.VideoCapture('/Users/analeegraig/Documents/Computer Vision/final_project_data/video.avi')
count_line_position = 1000

# minimum contour width
min_contour_width = 40

# minimum contour height
min_contour_height= 40

In [2]:
# other option

# store frames
ret, frame1 = cap.read()
ret, frame2 = cap.read()

# difference

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (5, 5), 0)
    
    _, thresh = cv2.threshold(blur, 80, 255, cv2.THRESH_BINARY)
    
    dilated = cv2.dilate(thresh, np.ones((3, 3)), iterations = 2)
    #dilated = cv2.erode(thresh, np.ones((3, 3)), iterations = 2)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilated2 = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 5000:
            continue
            
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame1, "Status: {}".format('Movement'), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
    
    # cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)
    
    cv2.imshow("inter", diff)
    #cv2.imshow("feed", frame1)
    
    frame1 = frame2
    ret, frame2 = cap.read()
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [3]:
# initial subtractor
algo = cv2.createBackgroundSubtractorMOG2()

# defining a function
def get_centroid(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy
    return [cx, cy]

detect = []
offset = 6
counter = 0

# Adding the following variables for speed calculation
fps_start_time = time.time()
fps = 0
total_frames = 0
speed_kph = 0
start_point = (0,0)

def calculate_speed(start_point, end_point, fps):
    distance_pixels = abs(end_point[0] - start_point[0])
    distance_meters = distance_pixels / pixels_per_meter 
    speed_mps = distance_meters / fps
    speed_kph = speed_mps * 3.6
    return speed_kph

pixels_per_meter = 10

while True:
    ret, frame1 = cap.read()
    
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (21, 21), 0)
    
    # applying on each frame
    img_sub = algo.apply(blur)

    # dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    # erode instead of dilate --> reduce bright areas of image
    dilat = cv2.erode(img_sub, np.ones((5, 5)))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    
    #kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    
    # dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    # dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_OPEN, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_OPEN, kernel)
    
    contourShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.line(frame1, (5, count_line_position), (1900, count_line_position), (255,127,0), 1)

    for contour in contourShape:
        if cv2.contourArea(contour) < 10000:
            continue
        
        else:
            for(i,c) in enumerate(contourShape):
                (x,y,w,h) = cv2.boundingRect(c)
                contour_valid = (w >= min_contour_width) and (h >= min_contour_height)
                if not contour_valid:
                    continue

                centroid = get_centroid(x, y, w, h)
                detect.append(centroid)
                cv2.circle(frame1,centroid, 4, (0,255,0), -1)

                # cx,cy= get_centroid(x, y, w, h)
                for (x, y) in detect:
                    if y < (count_line_position + offset) and y > (count_line_position - offset):
                        counter += 1
                        cv2.line(frame1, (5, count_line_position), (700, count_line_position), (0, 127, 255), 1)
                        detect.remove((x, y))
                        print("Vehicle Count: " + str(counter))

                        # Calculate speed
                        end_point = (x, y)
                        speed_kph = calculate_speed(start_point, end_point, fps)
                        print(f"Speed of vehicle {counter}: {speed_kph:.2f} km/h")

                        # BREAK AFTER 30 CARS
                        if counter > 30:
                            break
                            
                cv2.rectangle(frame1,(x,y),(x+w,y+h),(255,0,0),2)
                cv2.putText(frame1, "Vehicle " + str(counter), (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 244, 0), 2)

        if counter > 30:
                    break

        # Update fps and total frames
        total_frames += 1
        fps_end_time = time.time()
        fps = total_frames / (fps_end_time - fps_start_time)

        # Display speed and fps
        # cv2.putText(frame1, f"Speed: {speed_kph:.2f} km/h", (x, y - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # cv2.putText(frame1, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        #cv2.putText(frame1, "Status: {}".format(speed_kph), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)

    
    cv2.imshow('original', frame1)
    #cv2.imshow('Gray Video', dilatada)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    
    #if cv2.waitKey(1) == 13:
    #    break
    
cap.release()
cv2.destroyAllWindows()

Vehicle Count: 1
Speed of vehicle 1: 5.46 km/h
Vehicle Count: 2
Speed of vehicle 2: 5.45 km/h
Vehicle Count: 3
Speed of vehicle 3: 0.66 km/h
Vehicle Count: 4
Speed of vehicle 4: 0.66 km/h
Vehicle Count: 5
Speed of vehicle 5: 0.65 km/h
Vehicle Count: 6
Speed of vehicle 6: 18.13 km/h
Vehicle Count: 7
Speed of vehicle 7: 18.15 km/h
Vehicle Count: 8
Speed of vehicle 8: 17.86 km/h
Vehicle Count: 9
Speed of vehicle 9: 17.88 km/h
Vehicle Count: 10
Speed of vehicle 10: 15.71 km/h
Vehicle Count: 11
Speed of vehicle 11: 15.73 km/h
Vehicle Count: 12
Speed of vehicle 12: 16.78 km/h
Vehicle Count: 13
Speed of vehicle 13: 16.80 km/h
Vehicle Count: 14
Speed of vehicle 14: 11.59 km/h
Vehicle Count: 15
Speed of vehicle 15: 11.61 km/h
Vehicle Count: 16
Speed of vehicle 16: 11.55 km/h
Vehicle Count: 17
Speed of vehicle 17: 11.56 km/h
Vehicle Count: 18
Speed of vehicle 18: 11.51 km/h
Vehicle Count: 19
Speed of vehicle 19: 11.53 km/h
Vehicle Count: 20
Speed of vehicle 20: 9.49 km/h
Vehicle Count: 21
Speed 